# 03 - The Pattern Matcher

The next step is to progressively rewrite the UOp tree using the Pattern Matcher (PM).

The PM is used all over TinyGrad for different purposes, and I will cover it in greater detail later, but let's take a quick peek.

In [ ]:
from tinygrad.ops import PatternMatcher, UPat, graph_rewrite

The PM operates on a list of rules.

Each rule consists of a `UPat`, and a function that is called when the pattern matches part of the tree.

The return value of the function is the result of the "match".

In [ ]:
test_rules = PatternMatcher([
    (UPat(Ops.SINK), lambda: "U stink"),                                                # This rule matches any `SINK` UOp
    (UPat(Ops.CONST, name="x"), lambda x: f"Got a CONST dtype {x.dtype} arg {x.arg}"),  # Can pass the Op to the function
    (UPat(Ops.CONST), lambda x: f"Another rule for CONST"),                             # Oops, only one rule can match!
    (UPat((Ops.ADD, Ops.MUL)), lambda: "ADD or MUL"),                                   # Can match more than one UOp type
    (UPat(Ops.EXPAND, src=(UPat(Ops.RESHAPE, src=UPat(Ops.CONST, arg=2)))),
        lambda: "Expand with reshape from a const with arg=2")                          # Can match a specific sub-tree.
                                                                                        # Note: This one only matches the EXPAND for 2, not 1
    # No match - return Null
])

[test_rules.rewrite(op) for op in a_sink.toposort]

NameError: name 'a_sink' is not defined

A more interesting pattern is to replace the matched UOps with some other UOps. We can also use `graph_rewrite` to operate on a tree.

In [ ]:
insanity = PatternMatcher([
    (UPat(Ops.ADD, name="x"), lambda x: UOp(Ops.SUB, dtype=x.dtype, arg=x.arg, src=x.src)),
    (UPat(Ops.MUL, dtype=dtypes.ints, name="x"), lambda x: UOp(Ops.IDIV, dtype=x.dtype, src=x.src))
])

rewritten = graph_rewrite(add1, insanity)
rewritten

UOp(Ops.SUB, dtypes.int, arg=None, src=(
  UOp(Ops.IDIV, dtypes.int, arg=None, src=(
    UOp(Ops.CONST, dtypes.int, arg=3, src=()),
    UOp(Ops.CONST, dtypes.int, arg=5, src=()),)),
  UOp(Ops.CONST, dtypes.int, arg=2, src=()),))

In [ ]:
int(rewritten)

-2